[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GabbyTab/boofun/blob/main/notebooks/hw1_fourier_expansion.ipynb)

# CS 294-92: Analysis of Boolean Functions - Homework 1

## Fourier Expansion and Properties

This notebook accompanies Problem Set 1, exploring the Fourier expansion of Boolean functions.

**Instructor:** Avishay Tal  
**Reference:** O'Donnell, *Analysis of Boolean Functions*, Chapters 1-2
**Notebook by: Gabriel Taboada**

---

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../src')

import boofun as bf
from boofun.analysis import fourier

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("boofun loaded successfully")
print()
print("Quick API demo:")
print(f"  bf.parity(3).degree()       = {bf.parity(3).degree()}")
print(f"  bf.majority(3).influences() = {bf.majority(3).influences()}")

## Background: The Fourier Expansion

Every function $f: \{-1, 1\}^n \to \mathbb{R}$ has a unique **Fourier expansion**:

$$f(x) = \sum_{S \subseteq [n]} \hat{f}(S) \prod_{i \in S} x_i$$

where $\chi_S(x) = \prod_{i \in S} x_i$ are the **Fourier characters** (Walsh functions).

The **Fourier coefficients** are:
$$\hat{f}(S) = \mathbf{E}_{x \sim \{\pm 1\}^n}[f(x) \chi_S(x)]$$

Key properties:
- **Parseval's Identity:** $\mathbf{E}[f(x)^2] = \sum_S \hat{f}(S)^2$
- For Boolean functions $f: \{-1,1\}^n \to \{-1,1\}$: $\sum_S \hat{f}(S)^2 = 1$

---
## Problem 1: Fourier Transformations

Given $f(x) = \sum_{S \subseteq [n]} \hat{f}(S) \prod_{i \in S} x_i$, find the Fourier expansion of various transformations.

### Part (a): $g(x) = f(-x)$

When we negate all inputs, odd-degree coefficients flip sign: $\hat{g}(S) = (-1)^{|S|} \hat{f}(S)$

In [ ]:
# Let's explore g(x) = f(-x) using the Majority function
maj3 = bf.majority(3)  # Clean API: bf.majority() instead of bf.BooleanFunctionBuiltins.majority()
print("Original function: Majority-3")
print(f"Truth table: {list(maj3.get_representation('truth_table'))}")

# Compute Fourier coefficients directly on the function
coeffs = maj3.fourier()  # Clean: f.fourier() instead of SpectralAnalyzer(f).fourier_expansion()

print("\nFourier coefficients of f:")
for s in range(len(coeffs)):
    if abs(coeffs[s]) > 1e-10:
        subset = [i for i in range(3) if (s >> (2-i)) & 1]
        print(f"  f̂({subset}): {coeffs[s]:.4f}")

# Now compute g(x) = f(-x) using unary minus!
g = -maj3  # Clean: -f instead of fourier.negate_inputs(f)
coeffs_g = g.fourier()

print("\nFourier coefficients of g(x) = f(-x):")
for s in range(len(coeffs_g)):
    if abs(coeffs_g[s]) > 1e-10:
        subset = [i for i in range(3) if (s >> (2-i)) & 1]
        print(f"  ĝ({subset}): {coeffs_g[s]:.4f}")

print("\n💡 Observation: ĝ(S) = (-1)^|S| · f̂(S) (odd-degree coefficients flip sign)")

---
## Problem 2: Computing Fourier Expansions

### Part (a): MUX₃ (Multiplexer on 3 bits)

$\text{MUX}_3(x_1, x_2, x_3)$ outputs $x_2$ if $x_1 = 1$, and $x_3$ if $x_1 = -1$.

In [ ]:
# MUX₃: output x₂ if x₁=+1 (i.e., x₁=0 in {0,1}), else x₃
mux3_tt = [0, 0, 1, 1, 0, 1, 0, 1]
mux3 = bf.create(mux3_tt)

print("MUX₃ Truth Table:")
print("x₁ x₂ x₃ | MUX₃")
print("-" * 15)
for x in range(8):
    x1, x2, x3 = (x >> 2) & 1, (x >> 1) & 1, x & 1
    print(f" {x1}  {x2}  {x3} |   {mux3_tt[x]}")

# Compute and display Fourier expansion
mux3_coeffs = fourier.compute_mux3_fourier()
print("\nFourier Expansion of MUX₃:")
for s, coeff in sorted(mux3_coeffs.items()):
    subset = [i+1 for i in range(3) if (s >> (2-i)) & 1]
    print(f"  f̂({subset}): {coeff:.4f}")

print("\n📝 Mathematical derivation:")
print("   MUX₃(x) = (1+x₁)/2 · x₂ + (1-x₁)/2 · x₃")
print("          = x₂/2 + x₁x₂/2 + x₃/2 - x₁x₃/2")

### Part (b): NAE₃ (Not-All-Equal)

$\text{NAE}_3(x_1, x_2, x_3) = 1$ iff $x_1, x_2, x_3$ are not all equal.

In [ ]:
# NAE₃: output 1 if not all bits are equal
nae3_tt = [0, 1, 1, 1, 1, 1, 1, 0]
nae3 = bf.create(nae3_tt)

print("NAE₃ Truth Table:")
print("x₁ x₂ x₃ | NAE₃")
print("-" * 15)
for x in range(8):
    x1, x2, x3 = (x >> 2) & 1, (x >> 1) & 1, x & 1
    print(f" {x1}  {x2}  {x3} |   {nae3_tt[x]}")

# Compute Fourier expansion
nae3_coeffs = fourier.compute_nae3_fourier()
print("\nFourier Expansion of NAE₃:")
for s, coeff in sorted(nae3_coeffs.items()):
    subset = [i+1 for i in range(3) if (s >> (2-i)) & 1]
    print(f"  f̂({subset}): {coeff:.4f}")

### Part (c): AND_n

$\text{AND}_n(x) = 1$ iff all $x_i = 1$.

For $\text{AND}_n$ in $\{\pm 1\}$ notation, the Fourier coefficient is:
$$\hat{f}(S) = \frac{(-1)^{n-|S|}}{2^n}$$

In [ ]:
# AND_n for various n
for n in [2, 3, 4]:
    and_coeffs = fourier.compute_and_fourier(n)
    print(f"\nAND_{n} Fourier Expansion:")
    for s, coeff in sorted(and_coeffs.items())[:6]:  # Show first 6
        subset = [i+1 for i in range(n) if (s >> (n-1-i)) & 1]
        print(f"  f̂({subset}): {coeff:.6f}")
    if len(and_coeffs) > 6:
        print(f"  ... ({len(and_coeffs)} total coefficients)")
    
    # Verify formula
    expected = 1.0 / (2**n)
    print(f"  Expected magnitude: ±{expected:.6f} ✓")

---
## Problem 3: Parseval's Identity

**Statement:** For any $f: \{-1,1\}^n \to \mathbb{R}$:
$$\mathbf{E}[f(x)^2] = \sum_{S \subseteq [n]} \hat{f}(S)^2$$

For Boolean functions $f: \{-1,1\}^n \to \{-1,1\}$, both sides equal 1.

In [ ]:
# Verify Parseval's identity for various functions
# Using the clean API: bf.majority(), bf.parity(), etc.
test_functions = [
    ("XOR", bf.create([0, 1, 1, 0])),
    ("AND", bf.AND(2)),
    ("OR", bf.OR(2)),
    ("Majority-3", bf.majority(3)),
    ("Parity-3", bf.parity(3)),
]

print("Parseval's Identity Verification")
print("=" * 60)
print(f"{'Function':<15} {'Σf̂(S)²':>15} {'degree':>10} {'balanced':>10}")
print("-" * 60)

for name, f in test_functions:
    sum_sq = sum(c**2 for c in f.fourier())  # Parseval: should equal 1
    status = "✓" if abs(sum_sq - 1.0) < 1e-10 else "✗"
    print(f"{name:<15} {sum_sq:>15.6f} {f.degree():>10} {'yes' if f.is_balanced() else 'no':>10}")

print("\n💡 For Boolean functions f: {-1,1}^n → {-1,1}, Σf̂(S)² = 1 (Parseval)")

---
## Problem 4: Affine Function Testing

**Definition:** $f: \mathbb{F}_2^n \to \mathbb{F}_2$ is affine iff $f(x) = b + a_1 x_1 + \cdots + a_n x_n$ for some $a \in \mathbb{F}_2^n, b \in \mathbb{F}_2$.

**Characterization:** $f$ is affine iff $f(x) + f(y) + f(z) = f(x + y + z)$ for all $x, y, z$.

In [ ]:
# Test affine property on various functions using clean API
test_functions = [
    ("XOR (affine)", bf.create([0, 1, 1, 0])),
    ("x₁ (affine)", bf.dictator(2, 0)),
    ("AND (not affine)", bf.AND(2)),
    ("Majority-3", bf.majority(3)),
    ("Parity-3 (affine)", bf.parity(3)),
]

print("Affine Function Test")
print("=" * 60)

for name, f in test_functions:
    # Clean API: f.is_linear() and f.degree(gf2=True)
    is_affine = f.is_linear()
    gf2_deg = f.degree(gf2=True)  # GF(2) degree
    fourier_deg = f.degree()       # Fourier degree
    
    result = "✓ affine" if is_affine else "✗ not affine"
    print(f"{name:<20}: {result}, deg_GF2={gf2_deg}, deg_Fourier={fourier_deg}")

print("\n💡 Affine functions have GF(2) degree ≤ 1")
print("💡 Notice: XOR has GF(2) degree 1 but Fourier degree 2!")

---
## Problem 6: Degree-1 Functions

**Claim:** If $f: \{\pm 1\}^n \to \{\pm 1\}$ has Fourier degree at most 1, then $f$ is:
- A **constant** (f̂(∅) = ±1, all others 0), or
- A **dictator** (f(x) = x_i for some i), or
- An **anti-dictator** (f(x) = -x_i for some i)

In [ ]:
# Test degree-1 function classification using clean API
def classify_by_degree(f):
    """Classify a Boolean function by its Fourier degree."""
    deg = f.degree()  # Clean: f.degree() instead of fourier.fourier_degree(f)
    
    if deg == 0:
        return "Constant"
    if deg == 1:
        # Check which variable is the dictator
        coeffs = f.fourier()
        for i in range(f.n_vars):
            idx = 1 << (f.n_vars - 1 - i)
            if abs(coeffs[idx] - 1.0) < 1e-10:
                return f"Dictator x_{i}"
            if abs(coeffs[idx] + 1.0) < 1e-10:
                return f"Anti-dictator (NOT x_{i})"
        return "Degree-1"
    return f"Higher degree ({deg})"

# Test various functions using clean shortcuts
functions = [
    ("Constant 0", bf.constant(False, 3)),
    ("Constant 1", bf.constant(True, 3)),
    ("Dictator x₀", bf.dictator(3, 0)),
    ("Dictator x₂", bf.dictator(3, 2)),
    ("XOR", bf.parity(3)),
    ("AND", bf.AND(3)),
    ("Majority", bf.majority(3)),
]

print("Function Classification by Degree")
print("=" * 60)

for name, f in functions:
    deg = f.degree()
    W1 = f.W(1)  # Weight at degree 1
    classification = classify_by_degree(f)
    print(f"{name:<12}: degree={deg}, W^{{=1}}={W1:.4f}, → {classification}")

---
## Summary

In this notebook, we explored the problems from HW1:

1. **Problem 1 - Fourier Transformations**: How $f(-x)$, $f^{odd}$, $f^{even}$ relate to Fourier coefficients
2. **Problem 2 - Computing Expansions**: MUX₃, NAE₃, AND_n examples
3. **Problem 3 - Parseval's Identity**: $\mathbf{E}[f^2] = \sum_S \hat{f}(S)^2$
4. **Problem 4 - Affine Testing**: 4-query test based on $f(x) + f(y) + f(z) = f(x+y+z)$
5. **Problem 6 - Degree-1 Functions**: Always constant/dictator/anti-dictator

### Key Takeaways

- The **Fourier expansion** represents Boolean functions as multilinear polynomials
- **Parseval's identity** connects function values to Fourier coefficients
- **GF(2) degree** (algebraic) and **Fourier degree** can differ!
- The `boofun` library provides tools for all these computations

### Next Steps

- Try implementing the remaining problems from HW1
- Explore the spectral concentration of decision trees (Lecture 6)
- Investigate the connection to learning theory (LMN Theorem)